Codigo de prueba para encontrar a Wally
------------------------------------------------------------

Ojala lo hubiera tenido de chiquillo.


El ejemplo es muy sencillo, se coje el resultado de una escena donde esta wally. Se recorta el jersey por ser mas invariante a los cambios.

Se podria haber elegido la cabeza o el gorro solo, pero como la orientacion de la cabeza puede cambiar es mas eficaz usar el torso, que al ser un rectangulo en la misma posicion (siempre que wally este de pie) es mas invariante a la situacion.

Elementos necesarios

* una coleccion de escenas.
* el torso de wally extraido de una escena.




In [5]:
import numpy as np
import cv2 as cv
from  matplotlib import pyplot as plt
location= r'C:\Users\alber\Downloads\\'
escena = location +'wally.jpg'
escena2= location + "dondeEstaWally.jpg"
templateCabeza = location + 'wallyCabeza.JPG'
templateTorso = location + 'wallyTorso.JPG'

method = cv.TM_CCOEFF_NORMED#cv.TM_SQDIFF_NORMED


def resize(img, scale):
    "scale in percent"
    w = int(img.shape[1] * scale / 100)
    h = int(img.shape[0] * scale / 100)
    dim = (w,h)
    # resize image
    resized = cv.resize(img, dim, interpolation = cv.INTER_AREA)
    return resized

    
    


La funcion *resize* se va a usar para poder comparar una unica plantilla con muchas escenas, ya que la correlacion es sensible a la orientacion (por eso se elige el torso) y a las dimensiones.

In [7]:
def encuentraWally(escenaCpy, template, method):
    results=[]
    for scale in range(50,101,1):
        templ = resize(template,scale)
        result = cv.matchTemplate(escenaCpy,templ,method)
    
        cv.normalize(result,result,0,1,cv.NORM_MINMAX, -1)
        
        _minVal , _maxVal , minLoc, maxLoc = cv.minMaxLoc(result,None)
        
        if method == cv.TM_SQDIFF_NORMED:
            matchLoc = minLoc
        else:
            matchLoc = maxLoc
            
        if matchLoc not in results:
            results.append([matchLoc,templ.shape])
        
        
    for result,templ in results:
        cv.rectangle(escenaCpy,result,
                    (result[0] + templ[1],
                     result[1] + templ[0]),
                (0,0,0), 2,2,0)
        
    return escenaCpy 





Ahora pada cada *escena* en nuestra *lista de escenas* buscamos a wally

In [ ]:
template = cv.imread(templateTorso, cv.IMREAD_COLOR)
escenaImg = cv.imread(escena2,cv.IMREAD_COLOR)
escenaCpy = escenaImg.copy()

for escenaCpy in escenas:
    wally = encuentraWally(escenaCpy,template,method)
    cv.imshow("resultado",wally)
    
cv.waitKey(0)


Conclusion
------------------

El algoritmo arroja muchos falsos positivos porque la correlacion es muy sensible a la orientacion y la relacion de dimensiones entre el patron y la imagen. Quedaria pendiente comprobar si se puede utilizar el patron de colores del jersey de Wally (rojo,blanco) para mejorar el algorimo.